## Imports

In [55]:
import pandas as pd
import numpy as np
import json
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [6]:
#Load excel data
exp = pd.read_excel('../data/expenses_table.xlsx')

In [7]:
exp

,item,cadence,date_added,first_due,amount
0,Example (delete),12,2023-05-27,2023-06-30,60


In [8]:
#Check dtypes and update if needed
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   item        1 non-null      object        
 1   cadence     1 non-null      int64         
 2   date_added  1 non-null      datetime64[ns]
 3   first_due   1 non-null      datetime64[ns]
 4   amount      1 non-null      int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 168.0+ bytes


Nice! Good already.

Let's now calculate the next payment date. But first i'll add a count of the tracked months for reporting purposes.
https://stackoverflow.com/questions/30328427/add-months-to-a-datetime-column-in-pandas

In [9]:
#create a field that calculates the month in which you should've been saving since the last disbursement
exp['tracked_months'] = (((datetime.datetime.today().year-exp.date_added.dt.year)*12) + ((datetime.datetime.today().month-exp.date_added.dt.month))).astype(int)

In [10]:
exp

,item,cadence,date_added,first_due,amount,tracked_months
0,Example (delete),12,2023-05-27,2023-06-30,60,1


### Create a Last Disbursed Date Calculation

I'll do this with a while loop through each row

In [11]:
last_paid = []
for index, row in exp.iterrows():
    payment_date = row['first_due']
    if payment_date >= datetime.datetime.today():
        last_paid.append(np.nan) 
    else:
        #while ((datetime.datetime.today()-payment_date)/np.timedelta64(1, 'M')) > row['cadence']:
        while (((datetime.datetime.today().year-payment_date.dt.year)*12) + ((datetime.datetime.today().month-payment_date.dt.month))) > row['cadence']:
            payment_date += relativedelta(months=row['cadence'])
        last_paid.append(payment_date)

In [12]:
exp['last_disbursed']  = last_paid

In [13]:
#update dtype to datetime for future formula usage
exp['last_disbursed'] = pd.to_datetime(exp['last_disbursed'], format='%m,%d,%Y')

In [14]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT


### Create a Next Due Date Calculation

I'll do this with a while loop through each row

In [15]:
due_dates = []
for index, row in exp.iterrows():
    next_date = row['first_due']
    while next_date < datetime.datetime.today():
        next_date += relativedelta(months=row['cadence'])
    due_dates.append(next_date)

In [16]:
exp['due_next'] = due_dates

In [17]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30


Now I'll calculate the months between next payment

In [18]:
#exp['months_till_due'] = ((exp.due_next - datetime.datetime.today())/np.timedelta64(1, 'M')+1).astype(int)
exp['months_till_due'] = (((exp.due_next.dt.year-exp.date_added.dt.year)*12) + ((exp.due_next.dt.month-exp.date_added.dt.month))).astype(int)

In [19]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1


Now calculate the monthly savings. There will be a differnce though for the items in which we save for the whole cadence vs. those we add with less time than the cadence. so I'll flag it if we have a full 12 months (as an example) to save vs. only 3 months left of the 12 in which we started saving. I think we need a "months to save" field

In [20]:
#updating this to not if false use months till due, but to the lesser of the cadence or the recalculated months from starting tracking to having to pay. Adding +1 to include the starting month.
exp['months_to_save'] = np.where(exp['tracked_months'] >= exp['cadence'], exp['cadence'], (((exp.due_next.dt.year-exp.date_added.dt.year)*12) + ((exp.due_next.dt.month-exp.date_added.dt.month))+1).astype(int))

In [21]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2


In [22]:
exp['monthly_sinking'] = round(exp['amount'] / exp['months_to_save'],2)
exp.replace([np.inf, -np.inf], np.nan, inplace=True)

In [23]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0


Now I need to come up with a formula that calculates how much should be saved after your deposit this month based on how many months you should've saved since the last disbursement (or since you started tracking if no disbursement yet.)


In [24]:
#exp['current_buildup'] = np.where(exp['last_disbursed'] == 0, exp['tracked_months'], ((datetime.datetime.today()-exp.last_disbursed)/np.timedelta64(1, 'M')))
exp['current_buildup'] = np.where(exp['last_disbursed'].isnull(), exp['tracked_months'], (((datetime.datetime.today().year-exp.last_disbursed.dt.year)*12) + ((datetime.datetime.today().month-exp.last_disbursed.dt.month))))

In [25]:
#convert to an integer format so it will round down to get the exact # of months.
#using fillna here to make it possible to convert if there are any NaNs
exp['current_buildup'] = exp['current_buildup'].fillna(0).astype(int)


In [26]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0,1


Now we know how many months of savings we should have done since our last disbursement in each category so let's calculate what the total balance should be as of the last transfer to savings. I'll then calculate a final balance after transfer.

In [27]:
exp['exp_current_balance'] = exp['current_buildup'] * exp['monthly_sinking']

In [28]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0,1,30.0


Now that that's there, I want to create a column that calculates the disbursement (if there is one in the current month) as a negative value. I'll then combine that into a 'final_target' with expected_savings + monthly_needed - disbursement = final value.

In [29]:
exp['current_period_disburse'] = np.where((exp['due_next'].dt.month == datetime.datetime.today().month) & (exp['due_next'].dt.year == datetime.datetime.today().year), -exp['amount'], 0)

In [30]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0,1,30.0,-60


In [31]:
#sweet! Now I'll calculate the target ending balance
exp['target_ending_balance'] = exp['exp_current_balance'] + exp['monthly_sinking'] + exp['current_period_disburse']

In [32]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse,target_ending_balance
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0,1,30.0,-60,0.0


In [33]:
#one more column for indiviual item activity
exp['current_month_activity'] = exp['target_ending_balance'] - exp['exp_current_balance']

In [34]:
exp

,item,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse,target_ending_balance,current_month_activity
0,Example (delete),12,2023-05-27,2023-06-30,60,1,NaT,2023-06-30,1,2,30.0,1,30.0,-60,0.0,-30.0


### Aggregation Steps

Now that we have that all built out, we need some formulas that calculate the aggregate totals, and give some output to users after taking some inputs.

In [35]:
#I want a copy of this with a new cooler name
sinking_funds = exp.copy()

### Add ability to say which bills are due this month.

In [36]:
due_this_month = sinking_funds[sinking_funds['current_period_disburse'] < 0][['item','due_next','amount']]

In [37]:
due_dictionary = due_this_month.set_index('item').T.to_dict('list')

In [38]:
due_dictionary

{'Example (delete)': [Timestamp('2023-06-30 00:00:00'), 60]}

In [39]:
for key, value in due_dictionary.items():
    print(key, 'is due on', value[0].strftime('%m-%d-%Y'), ': $', str("{:.2f}".format(value[1])))

Example (delete) is due on 06-30-2023 : $ 60.00


### Setup way for user input of their actual current balance.

This comes in handy when there is interest building up. That is a little bit less you have to save. Or for times in which you fall behind by a couple of months.

In [40]:
#define a function to check for a valid float input
def valid_float(message):
    while True:
        try:
            count = float(input(message))
            if count >= 0:
                return count
                break
        except:
            print('You must input a numerical value.\n' + message)

In [41]:
actual_beg_balance = valid_float("Enter Current Sinking Funds Balance: ")

### Create Calculated Features and Setup Test Prints

In [42]:
expected_beginning_balance = sinking_funds['exp_current_balance'].sum()
current_period_savings = sinking_funds['monthly_sinking'].sum()
current_period_disbursements = sinking_funds['current_period_disburse'].sum()
net_period_activity = sinking_funds['current_month_activity'].sum()
target_ending_balance = sinking_funds['target_ending_balance'].sum()
actual_period_activity = target_ending_balance - actual_beg_balance
balance_adj = expected_beginning_balance - actual_beg_balance

In [43]:
print("You're expected current balance is: $" + str("{:.2f}".format(expected_beginning_balance)) + ".")
print("You're actual current balance is: $" + str("{:.2f}".format(actual_beg_balance)) + ".\n")
print('---'*10)
if actual_period_activity < 0:
    print("You should move $" + str("{:.2f}".format(-actual_period_activity)) + " from your sinking funds balance to your bill-paying account.")
else:
    print("You should move $" + str("{:.2f}".format(actual_period_activity)) + " from your checking account to your sinking funds balance.")
print("This is made up of: \n- $" + str("{:.2f}".format(current_period_savings)) + " savings for future expenses\n- $" + str("{:.2f}".format(-current_period_disbursements)) + " of disbursements for current month expense\n- $"+ str("{:.2f}".format(balance_adj)) + " of balance adjustments.\n")
print('---'*10)

print("With these changes, your target ending balance will be: $" + str("{:.2f}".format(target_ending_balance)) + ".")


You're expected current balance is: $30.00.
You're actual current balance is: $0.00.

------------------------------
You should move $0.00 from your checking account to your sinking funds balance.
This is made up of: 
- $30.00 savings for future expenses
- $60.00 of disbursements for current month expense
- $30.00 of balance adjustments.

------------------------------
With these changes, your target ending balance will be: $0.00.


In [44]:
print("-"*5 + "USER SUMMARY" + "-"*5)

-----USER SUMMARY-----


# V2 Testing
I want to remove excel from the equation and actually have this read/write from/to a json file. To do this we'll need to write functions for adding and removing records using user inputs.

Might pivot to a sqllite database for this. If I do go json, I either need to update the key and then have a subset with teh data. or have the title in the full layer with all the data and be able to add another line to this.


In [45]:
#establish possible inputs and alert
input_options = ['A','B','C','D']
alert = "Option not available."

In [50]:
def valid_select(message, n_choices):
    """Function for testing valid user input."""
    while True:
        try:
            answer = input(message).upper()
            if answer in input_options[:n_choices]:
                return answer
                break
            else:
                print(alert + ' Try again: ')
        except:
            print(alert + ' Try again: ')

In [54]:
valid_select("What would you like to do?\nA) View Current Month Report\nB) Add New Expense\nC) Delete an Expense", 4)

'A'

## Define Adding a new record

We'll need to first check if the selection exists, and then adjust from there.

### Test specifc reading

In [84]:
#test reading
with open('../data/exp.json', 'r') as f:
    data = json.load(f)

[item for item in data['expenses'] if item['title'] == 'Costco']

[{'title': 'Costco',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '6/30/2023',
  'amount': '60'}]

Might be able to use this code to get specific items
[d for d in a if d['name'] == 'pluto']


https://stackoverflow.com/questions/4391697/find-the-index-of-a-dict-within-a-list-by-matching-the-dicts-value

### Test Writing New Object

In [ ]:
#sample varible with new entry
new_entry = {'title': 'Xmas Gifts',
 'cadence': '12',
 'date_added': '5/27/2023',
 'first_due': '12/31/2023',
 'amount': '600'}

In [95]:
#test writing to json
with open('../data/exp.json') as f:
            data = json.load(f)
            temp = data["expenses"]
            temp.append(new_entry)
            with open('../data/exp.json', 'w') as f:
                json.dump(data, f)

In [100]:
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'Costco',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '6/30/2023',
  'amount': '60'},
 {'title': 'CC Fee',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '10/31/2023',
  'amount': '95'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

### Test updating specific value
As a test we'll update our Xmas Gift fund to $500 instead of $600.

In [102]:
#finding index #
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']
    index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)

index

2

In [105]:
#test saving down index # and writing to
with open('../data/exp.json') as f:
            data = json.load(f)
            temp = data["expenses"]
            index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)
            temp[index]['amount'] = 500
            with open('../data/exp.json', 'w') as f:
                json.dump(data, f)

In [106]:
#view if that worked
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'Costco',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '6/30/2023',
  'amount': '60'},
 {'title': 'CC Fee',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '10/31/2023',
  'amount': '95'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500}]

### Test Deleting Item